# 🌐 HTTP4s: Functional HTTP Services

**Phase 6: Libraries - Enterprise HTTP Integration**

**Master type-safe, functional HTTP services with HTTP4s - the purely functional HTTP library for Scala**

---

In [ ]:
// HTTP4s Functional HTTP Library
import cats.effect._
import cats.implicits._
import org.http4s._
import org.http4s.circe._
import org.http4s.circe.CirceEntityEncoder._
import org.http4s.circe.CirceEntityDecoder._
import org.http4s.server.blaze.BlazeServerBuilder
import org.http4s.client.blaze.BlazeClientBuilder
import scala.concurrent.ExecutionContext.global
import io.circe._
import io.circe.generic.auto._

println("🌐 HTTP4S: FUNCTIONAL HTTP SERVICES - TYPE-SAFE WEB DEVELOPMENT")
println()

println("🎯 HTTP4s Core Principles:")
println("✓ Purely functional HTTP server and client")
println("✓ Type-safe request/response handling")
println("✓ Middleware composition with cats")
println("✓ Streaming with fs2 integration")
println("✓ Built on cats-effect for async programming")
println()

// HTTP4s ecosystem
println("📊 HTTP4s Ecosystem:")
println("• Http4s-Core: Type classes and DSL")
println("• Http4s-Server: Server-side implementations")
println("• Http4s-Client: Client-side HTTP calls")
println("• Http4s-Circe: JSON serialization with Circe")
println("• Http4s-Blade: Fast, asynchronous server")
println()

## 📡 **HTTP Routes & Handlers**

**Type-safe route definitions and request handling with clean functional composition**

In [ ]:
// Domain models for our API
case class User(id: Long, name: String, email: String, age: Int)
case class CreateUserRequest(name: String, email: String, age: Int)
case class UserResponse(id: Long, name: String, email: String, age: Int, active: Boolean)
case class ErrorResponse(message: String, errorCode: String)

// HTTP Routes with functional composition
class UserRoutes[F[_]: Sync](userService: UserService[F]) {

  // Create User (POST /api/users)
  val createUser: HttpRoutes[F] = HttpRoutes.of[F] {
    case req @ POST -> Root / "api" / "users" =>
      for {
        createReq <- req.as[CreateUserRequest] // Automatic JSON decoding
        user <- userService.createUser(createReq.name, createReq.email, createReq.age)
        response <- Created(UserResponse(user.id, user.name, user.email, user.age, active = true))
      } yield response
  }

  // Get User by ID (GET /api/users/{id})
  val getUser: HttpRoutes[F] = HttpRoutes.of[F] {
    case GET -> Root / "api" / "users" / LongVar(userId) =>
      userService.getUser(userId).flatMap {
        case Some(user) =>
          Ok(UserResponse(user.id, user.name, user.email, user.age, active = true))
        case None =>
          NotFound(ErrorResponse("User not found", "USER_NOT_FOUND"))
      }
  }

  // List Users (GET /api/users?limit={limit}&offset={offset})
  val listUsers: HttpRoutes[F] = HttpRoutes.of[F] {
    case GET -> Root / "api" / "users" =>
      val limit = req.params.get("limit").flatMap(_.toIntOption).getOrElse(10)
      val offset = req.params.get("offset").flatMap(_.toIntOption).getOrElse(0)

      for {
        users <- userService.listUsers(limit, offset)
        userResponses = users.map(u => UserResponse(u.id, u.name, u.email, u.age, active = true))
        response <- Ok(userResponses)
      } yield response
  }

  // Delete User (DELETE /api/users/{id})
  val deleteUser: HttpRoutes[F] = HttpRoutes.of[F] {
    case DELETE -> Root / "api" / "users" / LongVar(userId) =>
      userService.deleteUser(userId).flatMap { deleted =>
        if (deleted) {
          NoContent()
        } else {
          NotFound(ErrorResponse("User not found", "USER_NOT_FOUND"))
        }
      }
  }

  // Health check endpoint (GET /health)
  val healthCheck: HttpRoutes[F] = HttpRoutes.of[F] {
    case GET -> Root / "health" =>
      Ok(Map("status" -> "healthy", "timestamp" -> System.currentTimeMillis().toString))
  }

  // Composite all routes - functional composition
  val routes: HttpRoutes[F] = createUser <+> getUser <+> listUsers <+> deleteUser <+> healthCheck
}

println("✅ HTTP4s Routes Implemented")
println("• Type-safe path parameters (LongVar, IntVar)")
println("• Automatic JSON encoding/decoding")
println("• Functional route composition (<+> operator)")
println("• Standard HTTP status codes (Ok, Created, NotFound, etc.)")
println("• Query parameter extraction from requests")
println()

## 🔐 **Middleware & Authentication**

**Reusable middleware for authentication, CORS, logging, and cross-cutting concerns**

In [ ]:
// HTTP4s Middleware Examples
import org.http4s.server.AuthMiddleware
import cats.data.{Kleisli, OptionT}
import scala.concurrent.duration._

// Authentication token and user
case class AuthUser(id: Long, username: String, roles: Set[String])
case class AuthToken(userId: Long, roles: Set[String], expiresAt: Long)

// Authentication service trait
trait AuthService[F[_]] {
  def validateToken(token: String): F[Option[AuthToken]]
  def getUser(token: AuthToken): F[Option[AuthUser]]
}

class JwtAuthService[F[_]: Sync] extends AuthService[F] {
  def validateToken(token: String): F[Option[AuthToken]] =
    Sync[F].pure {
      // Simplified token validation
      if (token.startsWith("Bearer ") && token.length > 50) {
        Some(AuthToken(123L, Set("user", "admin"), System.currentTimeMillis() + 3600000))
      } else None
    }

  def getUser(token: AuthToken): F[Option[AuthUser]] =
    Sync[F].pure(Some(AuthUser(token.userId, "alice@example.com", token.roles)))
}

// Authentication middleware using Kleisli
def authMiddleware[F[_]: Sync](authService: AuthService[F]): AuthMiddleware[F, AuthUser] = {
  val authUser: Kleisli[OptionT[F, *], Request[F], AuthUser] = Kleisli { req =>
    val tokenOpt = req.headers.get(ci"Authorization")
      .map(_.value)
      .filter(_.startsWith("Bearer "))
      .map(_.drop(7))

    tokenOpt match {
      case Some(token) =>
        OptionT(authService.validateToken(token).flatMap {
          case Some(authToken) => authService.getUser(authToken).map(_.map(_ => _))
          case None => Sync[F].pure(None)
        })
      case None => OptionT.none
    }
  }
  authUser
}

// CORS middleware for cross-origin requests
def corsMiddleware[F[_]: Sync]: HttpRoutes[F] => HttpRoutes[F] = { routes =>
  Kleisli { req =>
    val allowedMethods = Set("GET", "POST", "PUT", "DELETE", "OPTIONS")
    val allowedHeaders = Set("Accept", "Authorization", "Content-Type", "Origin")

    // Handle preflight OPTIONS requests
    if (req.method.name == "OPTIONS") {
      val response = Response[F](Status.Ok)
        .putHeaders(
          Header("Access-Control-Allow-Origin", "*"),
          Header("Access-Control-Allow-Methods", allowedMethods.mkString(", ")),
          Header("Access-Control-Allow-Headers", allowedHeaders.mkString(", "))
        )
      OptionT.liftF(Sync[F].pure(response))
    } else {
      // Add CORS headers to all responses
      OptionT(routes.run(req).value.map { respOpt =>
        respOpt.map(_.putHeaders(Header("Access-Control-Allow-Origin", "*")))
      })
    }
  }
}

// Request logging middleware
def loggingMiddleware[F[_]: Sync]: HttpRoutes[F] => HttpRoutes[F] = { routes =>
  Kleisli { req =>
    val start = System.currentTimeMillis()
    println(s"[REQUEST] ${req.method} ${req.uri} from ${req.remoteAddr.getOrElse("unknown")}")

    routes.run(req).value.map { respOpt =>
      respOpt.foreach { resp =>
        val duration = System.currentTimeMillis() - start
        println(f"[RESPONSE] ${resp.status.code} in ${duration}ms")
      }
      respOpt
    }
  }
}

println("🔐 HTTP4s Middleware Implemented")
println("• Authentication middleware with JWT validation")
println("• CORS headers for cross-origin requests")
println("• Request/response logging with timing")
println("• Middleware composition using Kleisli composition")
println()

## 🌐 **HTTP Client & External APIs**

**Type-safe HTTP clients for calling external services with proper error handling**

In [ ]:
// HTTP4s Client Examples
import org.http4s.client.Client
import java.net.URI

// External API response models
case class GitHubUser(login: String, id: Long, name: Option[String], followers: Int)
case class GitHubRepo(name: String, fullName: String, description: Option[String], language: Option[String])
case class WeatherResponse(temperature: Double, humidity: Double, description: String)

// Typed HTTP client for GitHub API
class GitHubClient[F[_]: Sync](client: Client[F]) {

  def getUser(username: String): F[Either[String, GitHubUser]] = {
    val uri = Uri.unsafeFromString(s"https://api.github.com/users/$username")
    val request = Request[F](Method.GET, uri)
      .putHeaders(Header("User-Agent", "Http4s-Client/1.0"))

    client.expect[GitHubUser](request).attempt.map {
      case Right(user) => Right(user)
      case Left(error) => Left(s"Failed to fetch GitHub user: ${error.getMessage}")
    }
  }

  def getUserRepos(username: String, perPage: Int = 30): F[Either[String, List[GitHubRepo]]] = {
    val uri = Uri.unsafeFromString(s"https://api.github.com/users/$username/repos")
      .withQueryParam("per_page", perPage.toString)
      .withQueryParam("sort", "updated")

    val request = Request[F](Method.GET, uri)
      .putHeaders(Header("User-Agent", "Http4s-Client/1.0"))

    client.expect[List[GitHubRepo]](request).attempt.map {
      case Right(repos) => Right(repos)
      case Left(error) => Left(s"Failed to fetch GitHub repos: ${error.getMessage}")
    }
  }
  
  def getRateLimit(): F[Either[String, Map[String, Int]]] = {
    val uri = Uri.unsafeFromString("https://api.github.com/rate_limit")
    val request = Request[F](Method.GET, uri)
      .putHeaders(Header("User-Agent", "Http4s-Client/1.0"))

    client.expect[Map[String, Int]](request).attempt.map {
      case Right(rateLimit) => Right(rateLimit)
      case Left(error) => Left(s"Failed to get rate limit: ${error.getMessage}")
    }
  }
}

// Weather API client example
class WeatherClient[F[_]: Sync](client: Client[F], apiKey: String) {

  def getCurrentWeather(city: String): F[Either[String, WeatherResponse]] = {
    val uri = Uri.unsafeFromString("https://api.weatherapi.com/v1/current.json")
      .withQueryParam("key", apiKey)
      .withQueryParam("q", city)
      .withQueryParam("aqi", "no")

    // Weather API returns complex JSON, we'll simulate a simple response
    val weather = WeatherResponse(72.5, 65.0, "Partly cloudy")
    Sync[F].pure(Right(weather))  // In real implementation, use client.expect
  }
}

// Resource-safe client usage
def withHttpClient[F[_]: Concurrent: Timer, A](f: Client[F] => F[A]): F[A] = {
  BlazeClientBuilder[F](global).resource.use(f)
}

// Example usage pattern
val githubWorkflow: IO[Unit] = withHttpClient { client =>
  val githubClient = new GitHubClient[IO](client)

  for {
    userResult <- githubClient.getUser("octocat")
    _ = userResult match {
      case Right(user) => println(s"Found GitHub user: ${user.login} (${user.followers} followers)")
      case Left(error) => println(s"GitHub API error: $error")
    }

    reposResult <- githubClient.getUserRepos("octocat", 5)
    _ = reposResult match {
      case Right(repos) => println(s"Found ${repos.length} repos")
      case Left(error) => println(s"GitHub repos error: $error")
    }

    rateLimit <- githubClient.getRateLimit()
    _ = rateLimit match {
      case Right(limits) => println(s"API rate limit: $limits")
      case Left(error) => println(s"Rate limit error: $error")
    }
  } yield ()
}

println("🌐 HTTP4s Client Implemented")
println("• Type-safe API clients with automatic JSON decoding")
println("• Query parameter composition")
println("• Resource-safe client lifecycle (auto-close connections)")
println("• Comprehensive error handling with Either types")
println("• REST API integration patterns")
println()

## 🚀 **Streaming & WebSocket Support**

**Real-time streaming responses and WebSocket bidirectional communication**

In [ ]:
// HTTP4s Streaming and WebSocket Support
import fs2.Stream
import fs2.concurrent.Queue
import org.http4s.websocket.WebSocketFrame
import org.http4s.websocket.WebSocketFrame._
import org.http4s.server.websocket.WebSocketBuilder
import scala.concurrent.duration._

println("🚀 HTTP4s Streaming & WebSocket Support")
println()

// Streaming response example (Server-Sent Events)
class StreamingRoutes[F[_]: Concurrent: Timer] {
  
  // Infinite stream of timestamped events
  val eventStream: Stream[F, String] = Stream
    .awakeEvery[F](1.second)
    .zipWithIndex
    .map { case (_, index) =>
      s"data: Event #${index + 1} at ${java.time.Instant.now()}\n\n"
    }

  // Server-Sent Events endpoint
  val sseRoute: HttpRoutes[F] = HttpRoutes.of[F] {
    case GET -> Root / "events" =>
      Ok(eventStream)
        .map(_.withContentType(`text/event-stream`))
  }

  // Large data streaming (file download simulation)
  val largeDataRoute: HttpRoutes[F] = HttpRoutes.of[F] {
    case GET -> Root / "download" / "large" =>
      val dataStream: Stream[F, Byte] = Stream
        .emits(List.range(0, 1000000)) // 1M items
        .map(_.toByte)
        .chunkN(4096) // Process in chunks
        .unchunks

      Ok(dataStream)
        .map(_.withContentType(`application/octet-stream`)
                .withHeader("Content-Disposition", "attachment; filename=\"large-data.bin\""))
  }

  // JSON stream processing
  val jsonStreamRoute: HttpRoutes[F] = HttpRoutes.of[F] {
    case GET -> Root / "api" / "users" / "stream" =>
      val users = List(
        UserResponse(1L, "Alice", "alice@example.com", 28, active = true),
        UserResponse(2L, "Bob", "bob@example.com", 32, active = true),
        UserResponse(3L, "Charlie", "charlie@example.com", 45, active = false)
      )

      val jsonStream: Stream[F, String] = Stream
        .emits(users)
        .map { user =>
          import io.circe.syntax._
          s"${user.asJson.noSpaces}\n" // JSON Lines format
        }
        .metered(500.millis) // Rate limit

      Ok(jsonStream)
        .map(_.withContentType(`application/x-ndjson`)) // Newline-delimited JSON
  }
}

// WebSocket chat room example
class WebSocketRoutes[F[_]: Concurrent] {

  // Shared broadcast queue for the chat room
  val chatQueue: Stream[F, Queue[F, WebSocketFrame]] = Stream.eval(Queue.unbounded[F, WebSocketFrame])
  val chatSystem: Stream[F, Unit] = chatQueue.flatMap { q =>
    Stream.constant(true).covary[F].flatMap { _ =>
      q.dequeue.flatMap { frame =>
        // Broadcast to all connected clients
        q.enqueue1(frame)
      }
    }
  }

  // WebSocket route
  val webSocketRoute: HttpRoutes[F] = HttpRoutes.of[F] {
    case GET -> Root / "ws" / "chat" =>
      val queue = Queue.unbounded[F, WebSocketFrame]

      val sendStream = Stream.empty.covary[F] // We don't send proactively
      val receiveSink = queue.dequeue.map { frame =>
        frame match {
          case Text(message, _) =>
            println(s"Received: $message")
            // In real implementation, broadcast to all connected clients
            ()
          case Close(_) =>
            println("WebSocket connection closed")
          case _ =>
            ()
        }
      }.drain

      WebSocketBuilder[F].build(send = sendStream, receive = receiveSink)
  }
}

println("📡 Streaming & WebSocket Features Implemented")
println("• Server-Sent Events for real-time updates")
println("• Large file streaming with chunked responses")
println("• JSON Lines streaming for API responses")
println("• WebSocket bidirectional communication")
println("• Rate-limited streaming with backpressure")
println()

println("🔧 Production Server Setup:")
println("object Http4sServer extends IOApp {")
println("  def run(args: List[String]): IO[ExitCode] = {")
println("    val routes = new UserRoutes[IO](userService)")
println("    val middleware = corsMiddleware(authMiddleware(routes.routes))")
println("    ")
println("    BlazeServerBuilder[IO](global)")
println("      .bindHttp(8080, \"0.0.0.0\")")
println("      .withHttpApp(middleware.orNotFound)")
println("      .serve")
println("      .compile")
println("      .drain")
println("      .as(ExitCode.Success)")
println("  }")
println("}")


## 🎯 **HTTP4s Best Practices**

**Production patterns and performance optimization techniques**

In [ ]:
// HTTP4s Production Patterns and Performance
println("🎯 HTTP4S PRODUCTION PATTERNS & PERFORMANCE OPTIMIZATION")
println()

// Error handling patterns
println("❌ Error Handling Patterns:")
println("• Use EitherT for application errors")
println("• Domain errors should be separate from technical errors")
println("• Map domain errors to appropriate HTTP status codes")
println()

println("⚡ Performance Optimization:")
println("• Use EntityDecoder for streaming large requests")
println("• Implement connection pooling for clients")
println("• Cache compiled routes for better startup time")
println("• Use streaming responses for large data")
println()

println("🔒 Security Best Practices:")
println("• Validate and sanitize all user inputs")
println("• Use HTTPS everywhere (TLS 1.3)")
println("• Implement proper CORS policies")
println("• Add rate limiting to protect resources")
println("• Log security events without sensitive data")
println()

println("🧪 Testing HTTP4s Applications:")
println("• Use http4s-dsl for test route construction")
println("• Mock services using trait-based design")
println("• Integration tests with real HTTP calls")
println("• Property-based testing for API contracts")
println()

println("📊 Monitoring Integration:")
println("• Add metrics with Prometheus + Micrometer")
println("• Distributed tracing with Jaeger/OpenTelemetry")
println("• Structured logging with correlation IDs")
println("• Health checks for load balancer integration")
println()

println("🏗️ Enterprise Architecture:")
println("• Layered architecture: Routes → Services → Repositories")
println("• Dependency injection with cats-effect Resource")
println("• Configuration management with pureconfig")
println("• Database transactions with doobie")
println("• Async processing with fs2 streaming")
println()

println("🎉 HTTP4s Mastered - Production-Ready Functional HTTP Services!")
println()

println("✨ Key Achievements:")
println("✓ Type-safe HTTP routing and request handling")
println("✓ Functional middleware composition")
println("✓ Automatic JSON encoding/decoding")
println("✓ Resource-safe client and server lifecycles")
println("✓ Streaming responses and WebSocket support")
println("✓ Enterprise error handling and security patterns")
println("✓ Production monitoring and observability integration")

println("\n🌟 HTTP4s revolutionizes web development with pure functional programming!")
